# CAPTCHA Recognition

In [7]:
import os
import numpy as np
from matplotlib import pyplot
from matplotlib import image
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

from PIL import Image

In [10]:
filenames = os.listdir('./archive')
labels_text = list(map(lambda x: x.split('.')[0], filenames))
to_tensor = transforms.ToTensor()
images = torch.tensor(
    [
        to_tensor(
            Image.open(f"./archive/{x}")
        ) for x in filenames
    ]
)
images.shape

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
index = 50
pyplot.imshow(images[index])
pyplot.show()
labels_text[index]

In [ ]:
import string

alphanumeric = {}

i = 0
while i < 10:
    onehot = [0] * 62
    onehot[i] = 1
    alphanumeric[str(i)] = onehot
    i += 1

for c in string.ascii_lowercase:
    onehot = [0] * 62
    onehot[i] = 1
    alphanumeric[c] = onehot
    i += 1

for c in string.ascii_uppercase:
    onehot = [0] * 62
    onehot[i] = 1
    alphanumeric[c] = onehot
    i += 1

labels = []
for l in labels_text:
    label = []
    for c in l:
        label += alphanumeric[c]
    labels.append(label)

labels = np.array(labels)
labels.shape


In [ ]:
class VGG19(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv9 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv14 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv15 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv16 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(4096, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 62 * 5)
    
    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = torch.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)

        out = torch.relu(self.conv3(out))
        out = torch.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)

        out = torch.relu(self.conv5(out))
        out = torch.relu(self.conv6(out))
        out = torch.relu(self.conv7(out))
        out = torch.relu(self.conv8(out))
        out = F.max_pool2d(out, 2)

        out = torch.relu(self.conv9(out))
        out = torch.relu(self.conv10(out))
        out = torch.relu(self.conv11(out))
        out = torch.relu(self.conv12(out))
        out = F.max_pool2d(out, 2)

        out = torch.relu(self.conv13(out))
        out = torch.relu(self.conv14(out))
        out = torch.relu(self.conv15(out))
        out = torch.relu(self.conv16(out))
        out = F.max_pool2d(out, 2)

        out = out.view(-1, 4096)

        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = torch.softmax(self.fc3(out))

        return out